In [ ]:
# Notebook 5: Model Interpretability for Amharic NER
# ==================================================

# Install required packages
!pip install shap lime transformers torch

import shap
import lime
from lime.lime_text import LimeTextExplainer
from transformers import AutoTokenizer, AutoModelForTokenClassification, pipeline

# ==============================
# 1. Load Trained Model
# ==============================
model_name = "./ner_model_final"  # Path from Notebook 3
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForTokenClassification.from_pretrained(model_name)

# Hugging Face NER pipeline
ner_pipeline = pipeline("ner", model=model, tokenizer=tokenizer, grouped_entities=True)

# ==============================
# 2. Example Message for Interpretation
# ==============================
example_text = "ሙሉ ቁርስ በ200 ብር በአዲስ አበባ"

# ==============================
# 3. SHAP Interpretation
# ==============================
# SHAP uses background dataset for reference
background_texts = ["የቤት እቃዎች ማስታወቂያ", "በ100 ብር እቃ ማስታወቂያ"]  # small sample
explainer = shap.Explainer(ner_pipeline, background_texts)

shap_values = explainer([example_text])
shap.plots.text(shap_values[0])

# ==============================
# 4. LIME Interpretation
# ==============================
class_names = ["O", "B-Product", "I-Product", "B-LOC", "I-LOC", "B-PRICE", "I-PRICE"]
explainer = LimeTextExplainer(class_names=class_names)

# Define prediction function for LIME
def predict_fn(texts):
    outputs = []
    for t in texts:
        ner_results = ner_pipeline(t)
        # Map NER entities to class probabilities (simplified)
        probs = [0] * len(class_names)
        for res in ner_results:
            if res['entity_group'] in class_names:
                idx = class_names.index(res['entity_group'])
                probs[idx] = res['score']
        outputs.append(probs)
    return np.array(outputs)

# Run LIME explanation
exp = explainer.explain_instance(example_text, predict_fn, num_features=10)
exp.show_in_notebook(text=True)
